In [ ]:
import os
from datetime import datetime
import glob
from collections import Counter
import pandas as pd
from typing import List, Counter
from tqdm import tqdm
import spacy
nlp = spacy.load('en_core_web_sm')
pd.set_option('display.max_rows', None)

### Variables

In [ ]:
#Paths
EUROPARL = '~/Dev/dissertation/Data/EUROPARLv3'
ENG = os.path.join(EUROPARL, 'english')
WHD = '~/Dev/dissertation/Data/WikipediaHomographData/data'
WHD_CTS = '~/Dev/dissertation/Data/WHD_CTS'

#Data
whd_df = pd.read_csv(os.path.join(WHD,'WikipediaHomographData.csv'))
europarl_whd_cts_df = whd_df.drop_duplicates(subset='homograph')

### Functions

In [ ]:
def get_tokens(PATH : str) -> List: 
    tokens = []
    nlp_file = nlp(' '.join(i), disable=['parser', 'tagger', 'ner']) 
    tokens = [token.lower_ for token in nlp_file if not token.is_punct]
    return tokens

In [ ]:
def get_whd_subset(token_ctr : Counter, europarl_whd_cts_df : pd.DataFrame) -> pd.DataFrame:
    europarl_whd_cts_df['cts'] = europarl_whd_cts_df['homograph'].apply(lambda hg : token_ctr[hg])
    europarl_whd_cts_df = europarl_whd_cts_df[['homograph', 'cts']]
    return europarl_whd_cts_df

## Script

10-21-20 Need to check if these English texts are repeated

In [ ]:
ttl_cts = []
for f in glob.glob(os.path.join(ENG, '*')):
    en_file = open(f, 'r', encoding='UTF-8').read().split(' ')
    n = 100000 #max allowed by spacy
    x = [en_file[i:i + n] for i in range(0, len(en_file), n)]
    tokens = []
    for i in tqdm(x):
        #Obtain lists of EUROPARL graphemes
        tokens.append(get_tokens(ENG))
    tokens =  [item for sublist in tokens for item in sublist]
    token_ctr = Counter(tokens)
    #Get subsets of Europarl graphemes also in Wikipedia Homograph Data with instance counts
    cts_df = get_whd_subset(token_ctr, europarl_whd_cts_df)

    #Serialize
    DATETIME = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    cts_df.to_csv(os.path.join(WHD_CTS, 'europarl_whd_{}_{}_cts.csv'.format(DATETIME, f[-8:-3])))

    #ttl_cts.append(all_cts_df)     
    
#Serialize
#ttl_cts_df = pd.concat(ttl_cts)
#DATETIME = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
#all_cts_df.to_csv(os.path.join(WHD_CTS, 'europarl_whd_{}_all_cts.csv'.format(DATETIME)))

In [ ]:
# #Review data
# cts_df = all_cts_df
# #Ct percents (~% Wikipedia graphemes have no instances in NXT SWBD; % have 1 instance)
# print(cts_df['cts'].value_counts(normalize=True) * 100)

# #Tokens with 40 greatest count values
# all_cts_df.sort_values(by=['cts'], ascending=False).head(40)

# #Tokens with 0 instances  (graphemes, ~% of WHs)
# zero_instances = cts_df[cts_df['cts'] == 0]
# print(zero_instances)
# print(zero_instances.shape)
# print(zero_instances.shape[0]/europarl_whd_cts_df.shape[0] * 100)
# #Tokens with 1-9 instances (graphemes, ~% of WHs)
# single_digit_instances = cts_df[(cts_df['cts'] > 0) & (cts_df['cts'] < 10)].sort_values(by=['cts'], ascending=False)
# print(single_digit_instances)
# print(single_digit_instances.shape)
# print(single_digit_instances.shape[0]/europarl_whd_cts_df.shape[0] *100)
# #Tokens with 10 or more instances (graphemes, ~% of WHs)
# more_instances = cts_df[cts_df['cts'] > 9].sort_values(by=['cts'], ascending=False)
# print(more_instances)
# print(more_instances.shape)
# print(more_instances.shape[0]/europarl_whd_cts_df.shape[0] *100)